In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats
from statsmodels.formula.api import ols
import xlsxwriter
from scipy.stats import linregress

In [2]:
# read in the new cleaned datasheet
df = pd.read_excel('..\Data Cleanup\cleaned_data.xlsx', sheet_name = 'master')

In [3]:
data = df[(df['EffMeas'] == 'LR') & (df['TotalApp'] == 1)]

In [4]:
columnList = ['Category', 'Loading', 'ConcDose', 'ClO2', 'MB', 'H2O2', 'VolApp', 'Temp', 'RH', 'ContTime', 
              'TotalApp', 'NumDataPointsLoading', 'NumDataPointsConcDose', 'NumDataPointsClO2', 'NumDataPointsMB', 
              'NumDataPointsH2O2', 'NumDataPointsVolApp', 'NumDataPointsTemp', 'NumDataPointsRH', 'NumDataPointsContTime',
              'NumDataPointsTotalApp', 'PLoading', 'PConc', 'PClO2', 'PMB', 'PH2O2', 'PVolApp', 'PTemp', 'PRH', 'PContTime', 
              'PTotalApp']

surfaceCats = ['IndoorInterior', 'IndoorExterior', 'IndoorCarpet', 'IndoorNonCarpet', 'IndoorCeilings', 'HVAC','IndoorMisc', 
               'OutdoorExterior', 'Roofing', 'Pavement', 'Water', 'Soil', 'OutdoorMisc', 'UndergroundInterior', 
               'UndergroundCarpet', 'UndergroundNonCarpet', 'UndergroundCeilings', 'UndergroundMisc']

methods = data.AppMethod.unique().tolist()

In [5]:
correlationData = pd.DataFrame(columns = columnList)

In [6]:
for surface in surfaceCats:
    for methodName in methods:
        title = " ".join([word.capitalize() for word in methodName.split(" ")])
        columnName = title + ' and ' + surface
        
        d1 = data[(data['AppMethod'] == methodName) & ((data['Indoor'].str.contains(surface)) | (data['Outdoor'].str.contains(surface)) | (data['Underground'].str.contains(surface)))]
        
        dLoading = d1[(d1['LoadingUn'] == 'CFU/cm^2')]
        
        p1 = pd.DataFrame()
        p1['Eff'] = dLoading.Eff
        p1['Loading'] = dLoading.Loading
        p1 = p1.dropna()
        
        dConcDose = d1[(d1['ConcDoseUn'] == "g/mL")]
        
        p2 = pd.DataFrame()
        p2['Eff'] = dConcDose.Eff
        p2['ConcDose'] = dConcDose.ConcDose
        p2 = p2.dropna()
        
        p3 = pd.DataFrame()
        p3['Eff'] = d1.Eff
        p3['ClO2'] = d1.ClO2
        p3 = p3.dropna()
        
        p4 = pd.DataFrame()
        p4['Eff'] = d1.Eff
        p4['MB'] = d1.MB
        p4 = p4.dropna()
        
        p5 = pd.DataFrame()
        p5['Eff'] = d1.Eff
        p5['H2O2'] = d1.H2O2
        p5 = p5.dropna()
        
        dVolApp = d1[(d1['VolAppUn'] == "g/cm^2")]
        
        p6 = pd.DataFrame()
        p6['Eff'] = dVolApp.Eff
        p6['VolApp'] = dVolApp.VolApp
        p6 = p6.dropna()
        
        p7 = pd.DataFrame()
        p7['Eff'] = d1.Eff
        p7['Temp'] = d1.Temp
        p7 = p7.dropna()
        
        p8 = pd.DataFrame()
        p8['Eff'] = d1.Eff
        p8['RH'] = d1.RH
        p8 = p8.dropna()
        
        p9 = pd.DataFrame()
        p9['Eff'] = d1.Eff
        p9['ContTime'] = d1.ContTime
        p9 = p9.dropna()
        
        p12 = pd.DataFrame()
        p12['Eff'] = d1.Eff
        p12['TotalApp'] = d1.TotalApp
        p12 = p12.dropna()
            
        rowData = [[columnName, 
                    stats.pearsonr(p1.Eff, p1.Loading)[0], 
                    stats.pearsonr(p2.Eff, p2.ConcDose)[0], 
                    stats.pearsonr(p3.Eff, p3.ClO2)[0], 
                    stats.pearsonr(p4.Eff, p4.MB)[0], 
                    stats.pearsonr(p5.Eff, p5.H2O2)[0], 
                    stats.pearsonr(p6.Eff, p6.VolApp)[0], 
                    stats.pearsonr(p7.Eff, p7.Temp)[0], 
                    stats.pearsonr(p8.Eff, p8.RH)[0], 
                    stats.pearsonr(p9.Eff, p9.ContTime)[0], 
                    stats.pearsonr(p12.Eff, p12.TotalApp)[0], 
                    min(p1.Eff.count(), p1.Loading.count()), 
                    min(p2.Eff.count(), p2.ConcDose.count()), 
                    min(p3.Eff.count(), p3.ClO2.count()), 
                    min(p4.Eff.count(), p4.MB.count()),
                    min(p5.Eff.count(), p5.H2O2.count()), 
                    min(p6.Eff.count(), p6.VolApp.count()), 
                    min(p7.Eff.count(), p7.Temp.count()), 
                    min(p8.Eff.count(), p8.RH.count()), 
                    min(p9.Eff.count(), p9.ContTime.count()),
                    min(p12.Eff.count(), p12.TotalApp.count()), 
                    stats.pearsonr(p1.Eff, p1.Loading)[1], 
                    stats.pearsonr(p2.Eff, p2.ConcDose)[1], 
                    stats.pearsonr(p3.Eff, p3.ClO2)[1], 
                    stats.pearsonr(p4.Eff, p4.MB)[1], 
                    stats.pearsonr(p5.Eff, p5.H2O2)[1], 
                    stats.pearsonr(p6.Eff, p6.VolApp)[1], 
                    stats.pearsonr(p7.Eff, p7.Temp)[1], 
                    stats.pearsonr(p8.Eff, p8.RH)[1], 
                    stats.pearsonr(p9.Eff, p9.ContTime)[1], 
                    stats.pearsonr(p12.Eff, p12.TotalApp)[1]]]
        
        dat = pd.DataFrame(rowData, columns = columnList)
            
        correlationData = pd.concat([dat, correlationData], ignore_index=True)

C:\Users\PERAZA\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\stats.py:3005: RuntimeWarning: Mean of empty slice.
  mx = x.mean()
C:\Users\PERAZA\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\PERAZA\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\stats.py:3006: RuntimeWarning: Mean of empty slice.
  my = y.mean()
C:\Users\PERAZA\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\stats.py:3010: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den
C:\Users\PERAZA\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\stats.py:5256: RuntimeWarning: invalid value encountered in less
  x = np.where(x < 1.0, x, 1.0)  # if x > 1 then return 1.0


In [7]:
for methodName in methods:
    title = " ".join([word.capitalize() for word in methodName.split(" ")])
    columnName = title
    d1 = data[(data['AppMethod'] == methodName)]
        
    dLoading = d1[(d1['LoadingUn'] == 'CFU/cm^2')]
        
    p1 = pd.DataFrame()
    p1['Eff'] = dLoading.Eff
    p1['Loading'] = dLoading.Loading
    p1 = p1.dropna()
        
    dConcDose = d1[(d1['ConcDoseUn'] == "g/mL")]
        
    p2 = pd.DataFrame()
    p2['Eff'] = dConcDose.Eff
    p2['ConcDose'] = dConcDose.ConcDose
    p2 = p2.dropna()
        
    p3 = pd.DataFrame()
    p3['Eff'] = d1.Eff
    p3['ClO2'] = d1.ClO2
    p3 = p3.dropna()
        
    p4 = pd.DataFrame()
    p4['Eff'] = d1.Eff
    p4['MB'] = d1.MB
    p4 = p4.dropna()
        
    p5 = pd.DataFrame()
    p5['Eff'] = d1.Eff
    p5['H2O2'] = d1.H2O2
    p5 = p5.dropna()
        
    dVolApp = d1[(d1['VolAppUn'] == "g/cm^2")]
        
    p6 = pd.DataFrame()
    p6['Eff'] = dVolApp.Eff
    p6['VolApp'] = dVolApp.VolApp
    p6 = p6.dropna()
        
    p7 = pd.DataFrame()
    p7['Eff'] = d1.Eff
    p7['Temp'] = d1.Temp
    p7 = p7.dropna()
        
    p8 = pd.DataFrame()
    p8['Eff'] = d1.Eff
    p8['RH'] = d1.RH
    p8 = p8.dropna()
        
    p9 = pd.DataFrame()
    p9['Eff'] = d1.Eff
    p9['ContTime'] = d1.ContTime
    p9 = p9.dropna()
        
    p12 = pd.DataFrame()
    p12['Eff'] = d1.Eff
    p12['TotalApp'] = d1.TotalApp
    p12 = p12.dropna()
            
    rowData = [[columnName, 
                stats.pearsonr(p1.Eff, p1.Loading)[0], 
                stats.pearsonr(p2.Eff, p2.ConcDose)[0], 
                stats.pearsonr(p3.Eff, p3.ClO2)[0], 
                stats.pearsonr(p4.Eff, p4.MB)[0], 
                stats.pearsonr(p5.Eff, p5.H2O2)[0], 
                stats.pearsonr(p6.Eff, p6.VolApp)[0], 
                stats.pearsonr(p7.Eff, p7.Temp)[0], 
                stats.pearsonr(p8.Eff, p8.RH)[0], 
                stats.pearsonr(p9.Eff, p9.ContTime)[0], 
                stats.pearsonr(p12.Eff, p12.TotalApp)[0], 
                min(p1.Eff.count(), p1.Loading.count()), 
                min(p2.Eff.count(), p2.ConcDose.count()), 
                min(p3.Eff.count(), p3.ClO2.count()), 
                min(p4.Eff.count(), p4.MB.count()),
                min(p5.Eff.count(), p5.H2O2.count()), 
                min(p6.Eff.count(), p6.VolApp.count()), 
                min(p7.Eff.count(), p7.Temp.count()), 
                min(p8.Eff.count(), p8.RH.count()), 
                min(p9.Eff.count(), p9.ContTime.count()),
                min(p12.Eff.count(), p12.TotalApp.count()),
                stats.pearsonr(p1.Eff, p1.Loading)[1], 
                stats.pearsonr(p2.Eff, p2.ConcDose)[1], 
                stats.pearsonr(p3.Eff, p3.ClO2)[1], 
                stats.pearsonr(p4.Eff, p4.MB)[1], 
                stats.pearsonr(p5.Eff, p5.H2O2)[1], 
                stats.pearsonr(p6.Eff, p6.VolApp)[1], 
                stats.pearsonr(p7.Eff, p7.Temp)[1], 
                stats.pearsonr(p8.Eff, p8.RH)[1], 
                stats.pearsonr(p9.Eff, p9.ContTime)[1], 
                stats.pearsonr(p12.Eff, p12.TotalApp)[1]]]
            
    dat = pd.DataFrame(rowData, columns = columnList)
            
    correlationData = pd.concat([dat, correlationData], ignore_index=True)

C:\Users\PERAZA\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\stats.py:3005: RuntimeWarning: Mean of empty slice.
  mx = x.mean()
C:\Users\PERAZA\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\PERAZA\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\stats.py:3006: RuntimeWarning: Mean of empty slice.
  my = y.mean()
C:\Users\PERAZA\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\stats.py:3010: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den
C:\Users\PERAZA\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\stats.py:5256: RuntimeWarning: invalid value encountered in less
  x = np.where(x < 1.0, x, 1.0)  # if x > 1 then return 1.0


In [8]:
for surface in surfaceCats:
    d1 = data[(data['Underground'].str.contains(surface)) | (data['Indoor'].str.contains(surface)) | (data['Outdoor'].str.contains(surface))]
    
    columnName = surface
    
    dLoading = d1[(d1['LoadingUn'] == 'CFU/cm^2')]
        
    p1 = pd.DataFrame()
    p1['Eff'] = dLoading.Eff
    p1['Loading'] = dLoading.Loading
    p1 = p1.dropna()
        
    dConcDose = d1[(d1['ConcDoseUn'] == "g/mL")]
        
    p2 = pd.DataFrame()
    p2['Eff'] = dConcDose.Eff
    p2['ConcDose'] = dConcDose.ConcDose
    p2 = p2.dropna()
        
    p3 = pd.DataFrame()
    p3['Eff'] = d1.Eff
    p3['ClO2'] = d1.ClO2
    p3 = p3.dropna()
        
    p4 = pd.DataFrame()
    p4['Eff'] = d1.Eff
    p4['MB'] = d1.MB
    p4 = p4.dropna()
        
    p5 = pd.DataFrame()
    p5['Eff'] = d1.Eff
    p5['H2O2'] = d1.H2O2
    p5 = p5.dropna()
        
    dVolApp = d1[(d1['VolAppUn'] == "g/cm^2")]
        
    p6 = pd.DataFrame()
    p6['Eff'] = dVolApp.Eff
    p6['VolApp'] = dVolApp.VolApp
    p6 = p6.dropna()
        
    p7 = pd.DataFrame()
    p7['Eff'] = d1.Eff
    p7['Temp'] = d1.Temp
    p7 = p7.dropna()
        
    p8 = pd.DataFrame()
    p8['Eff'] = d1.Eff
    p8['RH'] = d1.RH
    p8 = p8.dropna()
        
    p9 = pd.DataFrame()
    p9['Eff'] = d1.Eff
    p9['ContTime'] = d1.ContTime
    p9 = p9.dropna()
        
    p12 = pd.DataFrame()
    p12['Eff'] = d1.Eff
    p12['TotalApp'] = d1.TotalApp
    p12 = p12.dropna()
            
    rowData = [[columnName, 
                stats.pearsonr(p1.Eff, p1.Loading)[0], 
                stats.pearsonr(p2.Eff, p2.ConcDose)[0], 
                stats.pearsonr(p3.Eff, p3.ClO2)[0], 
                stats.pearsonr(p4.Eff, p4.MB)[0], 
                stats.pearsonr(p5.Eff, p5.H2O2)[0], 
                stats.pearsonr(p6.Eff, p6.VolApp)[0], 
                stats.pearsonr(p7.Eff, p7.Temp)[0], 
                stats.pearsonr(p8.Eff, p8.RH)[0], 
                stats.pearsonr(p9.Eff, p9.ContTime)[0], 
                stats.pearsonr(p12.Eff, p12.TotalApp)[0], 
                min(p1.Eff.count(), p1.Loading.count()), 
                min(p2.Eff.count(), p2.ConcDose.count()), 
                min(p3.Eff.count(), p3.ClO2.count()), 
                min(p4.Eff.count(), p4.MB.count()),
                min(p5.Eff.count(), p5.H2O2.count()), 
                min(p6.Eff.count(), p6.VolApp.count()), 
                min(p7.Eff.count(), p7.Temp.count()), 
                min(p8.Eff.count(), p8.RH.count()), 
                min(p9.Eff.count(), p9.ContTime.count()),
                min(p12.Eff.count(), p12.TotalApp.count()),
                stats.pearsonr(p1.Eff, p1.Loading)[1], 
                stats.pearsonr(p2.Eff, p2.ConcDose)[1], 
                stats.pearsonr(p3.Eff, p3.ClO2)[1], 
                stats.pearsonr(p4.Eff, p4.MB)[1], 
                stats.pearsonr(p5.Eff, p5.H2O2)[1], 
                stats.pearsonr(p6.Eff, p6.VolApp)[1], 
                stats.pearsonr(p7.Eff, p7.Temp)[1], 
                stats.pearsonr(p8.Eff, p8.RH)[1], 
                stats.pearsonr(p9.Eff, p9.ContTime)[1], 
                stats.pearsonr(p12.Eff, p12.TotalApp)[1]]]
            
    dat = pd.DataFrame(rowData, columns = columnList)
            
    correlationData = pd.concat([dat, correlationData], ignore_index=True)

C:\Users\PERAZA\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\stats.py:3010: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den
C:\Users\PERAZA\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\stats.py:5256: RuntimeWarning: invalid value encountered in less
  x = np.where(x < 1.0, x, 1.0)  # if x > 1 then return 1.0
C:\Users\PERAZA\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\stats.py:3005: RuntimeWarning: Mean of empty slice.
  mx = x.mean()
C:\Users\PERAZA\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\PERAZA\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\stats.py:3006: RuntimeWarning: Mean of empty slice.
  my = y.mean()


In [9]:
correlationData.to_excel("pearsonpv_correlation.xlsx", sheet_name='master')

In [10]:
cLoading = pd.DataFrame()
cLoading['Subset'] = correlationData.Category
cLoading['Loading'] = correlationData.Loading
cLoading['DataPoints'] = correlationData.NumDataPointsLoading
cLoading['PValue'] = correlationData.PLoading

cConcDose = pd.DataFrame()
cConcDose['Subset'] = correlationData.Category
cConcDose['ConcDose'] = correlationData.ConcDose
cConcDose['DataPoints'] = correlationData.NumDataPointsConcDose
cConcDose['PValue'] = correlationData.PConc

cClO2 = pd.DataFrame()
cClO2['Subset'] = correlationData.Category
cClO2['ClO2'] = correlationData.ClO2
cClO2['DataPoints'] = correlationData.NumDataPointsClO2
cClO2['PValue'] = correlationData.PClO2

cMB = pd.DataFrame()
cMB['Subset'] = correlationData.Category
cMB['MB'] = correlationData.MB
cMB['DataPoints'] = correlationData.NumDataPointsMB
cMB['PValue'] = correlationData.PMB

cH2O2 = pd.DataFrame()
cH2O2['Subset'] = correlationData.Category
cH2O2['H2O2'] = correlationData.H2O2
cH2O2['DataPoints'] = correlationData.NumDataPointsH2O2
cH2O2['PValue'] = correlationData.PH2O2

cVolApp = pd.DataFrame()
cVolApp['Subset'] = correlationData.Category
cVolApp['VolApp'] = correlationData.VolApp
cVolApp['DataPoints'] = correlationData.NumDataPointsVolApp
cVolApp['PValue'] = correlationData.PVolApp

cTemp = pd.DataFrame()
cTemp['Subset'] = correlationData.Category
cTemp['Temp'] = correlationData.Temp
cTemp['DataPoints'] = correlationData.NumDataPointsTemp
cTemp['PValue'] = correlationData.PTemp

cRH = pd.DataFrame()
cRH['Subset'] = correlationData.Category
cRH['RH'] = correlationData.RH
cRH['DataPoints'] = correlationData.NumDataPointsRH
cRH['PValue'] = correlationData.PRH

cContTime = pd.DataFrame()
cContTime['Subset'] = correlationData.Category
cContTime['ContTime'] = correlationData.ContTime
cContTime['DataPoints'] = correlationData.NumDataPointsContTime
cContTime['PValue'] = correlationData.PContTime

cTotalApp = pd.DataFrame()
cTotalApp['Subset'] = correlationData.Category
cTotalApp['TotalApp'] = correlationData.TotalApp
cTotalApp['DataPoints'] = correlationData.NumDataPointsTotalApp
cTotalApp['PValue'] = correlationData.PTotalApp

In [11]:
pVal = 0.01
cVal = 0.50

In [12]:
cLoading = cLoading[(cLoading['PValue'] <= pVal) & (abs(cLoading['Loading']) >= cVal) & (cLoading['DataPoints'] > 2)]
cConcDose = cConcDose[(cConcDose['PValue'] <= pVal) & (abs(cConcDose['ConcDose']) >= cVal) & (cConcDose['DataPoints'] > 2)]
cClO2 = cClO2[(cClO2['PValue'] <= pVal) & (abs(cClO2['ClO2']) >= cVal) & (cClO2['DataPoints'] > 2)]
cMB = cMB[(cMB['PValue'] <= pVal) & (abs(cMB['MB']) >= cVal) & (cMB['DataPoints'] > 2)]
cH2O2 = cH2O2[(cH2O2['PValue'] <= pVal) & (abs(cH2O2['H2O2']) >= cVal) & (cH2O2['DataPoints'] > 2)]
cVolApp = cVolApp[(cVolApp['PValue'] <= pVal) & (abs(cVolApp['VolApp']) >= cVal) & (cVolApp['DataPoints'] > 2)]
cTemp = cTemp[(cTemp['PValue'] <= pVal) & (abs(cTemp['Temp']) >= cVal) & (cTemp['DataPoints'] > 2)]
cRH = cRH[(cRH['PValue'] <= pVal) & (abs(cRH['RH']) >= cVal) & (cRH['DataPoints'] > 2)]
cContTime = cContTime[(cContTime['PValue'] <= pVal) & (abs(cContTime['ContTime']) >= cVal) & (cContTime['DataPoints'] > 2)]
cTotalApp = cTotalApp[(cTotalApp['PValue'] <= pVal) & (abs(cTotalApp['TotalApp']) >= cVal) & (cTotalApp['DataPoints'] > 2)]

In [13]:
with pd.ExcelWriter('pearsonpv_breakdown.xlsx') as writer:  
    if cLoading.shape[0] > 0:
        cLoading.to_excel(writer, sheet_name='Loading')
    
    if cConcDose.shape[0] > 0:
        cConcDose.to_excel(writer, sheet_name='ConcDose')
    
    if cClO2.shape[0] > 0:
        cClO2.to_excel(writer, sheet_name='ClO2')
    
    if cMB.shape[0] > 0:
        cMB.to_excel(writer, sheet_name='MB')
    
    if cH2O2.shape[0] > 0:
        cH2O2.to_excel(writer, sheet_name='H2O2')
    
    if cVolApp.shape[0] > 0:
        cVolApp.to_excel(writer, sheet_name='VolApp')
    
    if cTemp.shape[0] > 0:
        cTemp.to_excel(writer, sheet_name='Temp')
    
    if cRH.shape[0] > 0:
        cRH.to_excel(writer, sheet_name='RH')
    
    if cContTime.shape[0] > 0:
        cContTime.to_excel(writer, sheet_name='ContTime')
    
    if cTotalApp.shape[0] > 0:
        cTotalApp.to_excel(writer, sheet_name='TotalApp')

In [14]:
temp = data[(data['AppMethod'] == 'foam spray') & (data['Pavement'] == 0) & (data['Roofing'] == 0) & 
            (data['OutdoorExterior'] == 0) & (data['HVAC'] == 0) & (data['IndoorExterior'] == 0)]

if abs(stats.pearsonr(temp.Eff, temp.Temp)[0]) >= cVal and stats.pearsonr(temp.Eff, temp.Temp)[1] <= pVal:
    print('True')
    print(stats.pearsonr(temp.Eff, temp.Temp)[0])
    print(stats.pearsonr(temp.Eff, temp.Temp)[1])
    print(temp.Eff.count())

True
0.6591037525317794
1.0800325697842166e-07
52
